# ERA5


## Introduction

Exemple de calcul distribué sur le dataset ERA5 avec le stack de Pangeo

## Environnement

In [32]:
# Display output of plots directly in Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import intake
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.xarray
import s3fs
import pandas as pd
import geopandas as gpd
import hydrotools as ht
import hvplot.pandas

## Dask Distributed Cluster

In [2]:
# Create cluster
from dask_gateway import Gateway
from dask.distributed import Client
# gateway = Gateway()
# cluster = gateway.new_cluster()
# cluster.adapt(minimum=2, maximum=100)
# # Connect to cluster
# client = Client(cluster)
client = Client()

# Display cluster dashboard URL
# cluster

client

Client Scheduler: tcp://127.0.0.1:32907 Dashboard: http://127.0.0.1:33303/status,Cluster Workers: 4 Cores: 12 Memory: 33.48 GB



☝️ Link to scheduler dashboard will appear above.

## Prenons un bassin versant au hasard (polygone)

In [51]:
lst_shp = ['s3://watersheds-polygons/MELCC/json/011201/011201.json']
client_kwargs={"endpoint_url":"https://s3.us-east-2.wasabisys.com"}

s3 = s3fs.S3FileSystem(client_kwargs=client_kwargs,
                               anon=True)  # public read
gdf = pd.concat([gpd.GeoDataFrame.from_file(s3.open(file), encoding='latin-1')
                 for file in lst_shp])
gdf

In [54]:
gdf.hvplot(geo=True, 
           tiles='ESRI',
           xlim=(-60,-80),
           width=900,
           height=400, 
           color='y',
           hover_cols=['Station','Superficie'])

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Station,Superficie)

## Load data into xarray from a catalog using intake-esm


In [6]:
import intake
cat_url = 'https://raw.githubusercontent.com/hydrologie/datasets/main/intake-catalogs/atmosphere.yml'
cat = intake.open_catalog(cat_url)
cat.gui

In [ ]:
ds = cat.era5_hourly_reanalysis_land.to_dask()
ds

In [59]:
%%time

df_era5 = ht.clip_polygon_to_dataframe(dataset=ds,
                                       geodataframe=gdf,
                                       geodf_index_column='Station',
                                       variable='sd',  
                                       aggregation='mean',
                                       resample_time='1D', # originalement au pas de temps horaire
                                       from_tz='UTC',
                                       to_tz='UTC',
                                       latlng_names=['latitude',
                                                     'longitude']
                                       )
df_era5 = df_era5*1000

011201
CPU times: user 917 ms, sys: 86.1 ms, total: 1 s
Wall time: 7.5 s


In [64]:
df_era5['year'] = df_era5.index.year
df_era5.hvplot(x='time',
               width=900,
               grid=True, 
               ylim=(0,500),
               ylabel='EEN (mm)')

Column
    [0] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] DiscreteSlider(formatter='%d', margin=(20, 20, 20, 20), name='year', options=OrderedDict([('1981', ...]), value=1981, value_throttled=1981, width=250)
        [2] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top')